In [1]:
# Se instala la versión de "requests>=2.31.0", así mantenemos flexibilidad y 
# seguridad sin arriesgar cambios mayores incompatibles en una futura versión 3.x.

In [2]:
!pip install requests>=2.31.0

In [3]:
# --- Importar librerias ---

import requests
import base64
import time
import pandas as pd

In [4]:
# --- Credenciales Spotify ---

CLIENT_ID = 'Id de la App que se crea en la API de Spotify'
CLIENT_SECRET = 'Id de la App que se crea en la API de Spotify'

# --- Parámetros ---
# --- La API tiene un limite de solicitudes por segundos, por lo que en la variable "paises" se tienen que incorporar el pais que se quiere buscar ---

PAISES = ['BR', 'PT']
PALABRAS_CLAVE = ['technology', 'comedy', 'news', 'sports']  # en inglés, para mejorar resultados
LIMITE_SHOWS = 20


In [5]:
# Idioma esperado según país

Idioma_Por_Pais = {
    'AR': 'es',   # Argentina - Español
    'ES': 'es',   # España - Español
    'US': 'en',   # Estados Unidos - Inglés
    'GB': 'en',   # Reino Unido (Inglaterra) - Inglés
    'BR': 'pt',   # Brasil - Portugués
    'PT': 'pt',   # Portugal - Portugués
}


In [6]:
# --- Obtener token ---

def obtener_token(client_id, client_secret):
    auth_url = 'https://accounts.spotify.com/api/token'
    auth_header = base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()
    headers = {
        'Authorization': f'Basic {auth_header}',
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    data = {'grant_type': 'client_credentials'}
    res = requests.post(auth_url, headers=headers, data=data)
    if res.status_code != 200:
        raise Exception(f"Error autenticando: {res.status_code} - {res.text}")
    return res.json().get('access_token')
    

In [7]:
# --- Buscar shows por palabra clave (nombre del podcast) ---

def buscar_shows_por_keyword(keyword, pais, token, limite=LIMITE_SHOWS):
    url = f"https://api.spotify.com/v1/search"
    headers = {"Authorization": f"Bearer {token}"}
    params = {
        "q": keyword,
        "type": "show",
        "market": pais,
        "limit": limite
    }
    res = requests.get(url, headers=headers, params=params)
    if res.status_code != 200:
        print(f"⚠️ Error al buscar shows para '{keyword}' en {pais}: {res.status_code}")
        return []
    return res.json().get("shows", {}).get("items", [])
    

In [8]:
# --- Obtener episodios ---

def obtener_episodios(show_id, token):
    episodios = []
    url = f"https://api.spotify.com/v1/shows/{show_id}/episodes?market=US&limit=50"
    headers = {'Authorization': f'Bearer {token}'}
    while url:
        res = requests.get(url, headers=headers)
        if res.status_code == 429:
            retry_after = int(res.headers.get('Retry-After', 1))
            print(f"⏳ Rate limit. Esperando {retry_after} segundos...")
            time.sleep(retry_after)
            continue
        elif res.status_code != 200:
            print(f"⚠️ No se pudieron obtener episodios para show {show_id}: {res.status_code}")
            break
        data = res.json()
        episodios.extend(data.get('items', []))
        url = data.get('next')
        time.sleep(0.3)
    return episodios
    

In [9]:
# --- Formatear datos ---

def filtrar_y_formatear(show, episodios, keyword, pais):
    registros = []
    idioma_esperado = Idioma_Por_Pais.get(pais, 'es')
    idioma_show = show.get('languages', [''])[0]
    if idioma_show != idioma_esperado:
        return []

    # Filtramos los episodios válidos primero
    episodios_validos = []
    for ep in episodios:
        if not ep:
            continue
        fecha = ep.get('release_date')
        if fecha:
            try:
                año = int(fecha[:4])
                if 2020 <= año <= 2024:
                    episodios_validos.append(ep)
            except:
                continue

    # Cuántos episodios válidos fueron encontrados
    cantidad_descargados = len(episodios_validos)

    # Generamos registros con esa cantidad
    for ep in episodios_validos:
        registros.append({
            'pais': pais,
            'idioma_show': idioma_show,
            'keyword': keyword,
            'podcast_id': show['id'],
            'nombre_podcast': show['name'],
            'editorial': show.get('publisher'),
            'total_episodios': show.get('total_episodes'),
            'episodios_descargados': cantidad_descargados,
            'episodio_id': ep['id'],
            'nombre_episodio': ep['name'],
            'fecha_lanzamiento': fecha,
            'duracion_min': round(ep['duration_ms'] / 60000, 2)
        })

    return registros
    

In [ ]:
# --- Main ---

def main():
    token = obtener_token(CLIENT_ID, CLIENT_SECRET)
    datos = []
    for pais in PAISES:
        for palabra_clave in PALABRAS_CLAVE:
            print(f"🔍 Buscando podcasts relacionados con '{palabra_clave}' en {pais}...")
            shows = buscar_shows_por_keyword(palabra_clave, pais, token)
            for show in shows:
                episodios = obtener_episodios(show['id'], token)
                registros = filtrar_y_formatear(show, episodios, palabra_clave, pais)
                datos.extend(registros)

    df = pd.DataFrame(datos)
    df.to_csv('spotify_podcasts_keywords_2020_2024.csv', index=False)
    print(f"\n✅ Base de datos guardada con {len(df)} filas")

main()


🔍 Buscando podcasts relacionados con 'technology' en BR...
🔍 Buscando podcasts relacionados con 'comedy' en BR...
🔍 Buscando podcasts relacionados con 'news' en BR...
